# Assignment 1

## Question `2` (Decision Trees)

| | |
|-|-|
| Course | Statistical Methods in AI |
| Release Date | `19.01.2023` |
| Due Date | `29.01.2023` |

This assignment will have you working and experimenting with decision trees. Initially, you will be required to implement a decision tree classifier by choosing thresholds based on various impurity measures and reporting the scores. Later, you can experiment with the `scikit-learn` implementation of decision trees, and how various other parameters can be leveraged for better performance.

The dataset is a very simple one, the [banknote authentication dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication). It has 5 columns, the first 4 are the features, and the last one is the class label. The features are the variance, skewness, curtosis and entropy of the [wavelet transformed](https://en.wikipedia.org/wiki/Wavelet_transform) image of the banknote. The class label is 1 if the banknote is authentic, and 0 if it is forged. The data is present in `bankAuth.txt`. There are a total of 1372 samples in the dataset.

### Imports

In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# additional imports if necessary

### Impurity Measures

Decision trees are only as good as the impurity measure used to choose the best split. In this section, you will be required to implement the following impurity measures and use them to build a decision tree classifier.

1. Gini Index
2. Entropy/Log loss
3. Misclassification Error

Write functions that calculate the impurity measures for a given set of labels. The functions should take in a list of labels and return the impurity measure.

In [69]:
# your code here

def giniIndex(items):
    zeroCount = 0
    oneCount = 0
    for item in items:
        if item[-1] == 0:
            zeroCount += 1
        else:
            oneCount += 1

    N = float(items.shape[0])
    if N == 0:
        return 0
        
    oneFreq = oneCount/N
    zeroFreq = zeroCount/N

    gini = oneFreq*(1.0 - oneFreq) * zeroFreq*(1.0 - zeroFreq)
    return gini

def entropy(items):
    zeroCount = 0
    oneCount = 0
    for item in items:
        if item[-1] == 0:
            zeroCount += 1
        else:
            oneCount += 1

    N = float(items.shape[0])
    if N == 0:
        return 0

    oneFreq = oneCount/N
    zeroFreq = zeroCount/N

    S = -(np.log(oneFreq) * oneFreq + np.log(zeroFreq)*zeroFreq) 
    return S

def misclassificationError(items):
    zeroCount = 0
    oneCount = 0
    for item in items:
        if item[-1] == 0:
            zeroCount += 1
        else:
            oneCount += 1

    N = float(items.shape[0])
    if N == 0:
        return 0

    oneFreq = oneCount/N
    zeroFreq = zeroCount/N

    if oneFreq > zeroFreq:
        msce = 1 - oneFreq
    else:
        msce = 1 - zeroFreq
    return msce

### Decision Tree

Fit a decision tree using any one of the above impurity measures with a depth of 3. This means you will have eight leaf nodes and seven internal nodes. Report the threshold values at each internal node and the impurity measure at the final leaf node with the label. Also report the accuracy of the classifier on the training and test data (instructions for splitting the data will be given in the end).

In [124]:
# your code here
class Partition():
    def __init__(self, featureNum: int, value: float):
        self.featureNum = featureNum
        self.value = value

        if featureNum >= 4:
            print("Warning: Invalid feature num")

    # pass a np array of items
    def __call__(self, items): 
        # False -> left node, True -> right node
        result = items[:, self.featureNum] < self.value

        falseItems = items[np.where(result == False)]
        trueItems = items[np.where(result == True)]

        return [falseItems, trueItems]


class DecisionTree():
    """
    Letters are groups of values, numbers are partitions

                A
                0
           B          C
           1          2
        C     D    E     F
        3     4    5     6
      G   H  I J  K  L  M  N

    """
    
    def __init__(self, trainData, impurity=giniIndex):
        self.partitions = [Partition(-1,0.0)] * 7
        self.impurity = impurity
        self.trainData = trainData
        self.leafNodes = []

    # impurity for a specific partition
    def getImpurity(self, items, partition : Partition):
        falseItems, trueItems = partition(items)
        totalImpurity = self.impurity(falseItems) + self.impurity(trueItems)

        return totalImpurity
        
    # return the value that minimises impurity and impurity itself, for some feature
    def findBestValue(self, items, featureNum, start=-3.0, end=3.0, stepSize=0.1):
        lowestImpurity = np.inf
        bestCandidate = float(start+end)/2

        candidateValues = np.arange(start, end, stepSize)

        for candidate in candidateValues:
            p = Partition(featureNum, candidate)
            impurity = self.getImpurity(items, p)

            # print(candidate, impurity)

            if impurity < lowestImpurity:
                lowestImpurity = impurity
                bestCandidate = candidate

        print("Feature: ", featureNum, " | Best candidate: ", bestCandidate, " | Impurity: ", lowestImpurity)
        return bestCandidate, lowestImpurity

    # compare best choices from all features to greedily select a partition choice
    def findBestPartition(self, items, stepSize=0.025):
        numFeatures = items.shape[1] - 1
        impurities = np.empty((numFeatures, 2))

        for featureNum in range(numFeatures):
            minValue = np.min(items[:, featureNum])
            maxValue = np.max(items[:, featureNum])

            bestCandidate, lowestImpurity = self.findBestValue(items, featureNum, minValue, maxValue, stepSize)
            impurities[featureNum, :] = [bestCandidate, lowestImpurity]

        chosenFeature = np.argmin(impurities[:,1])
        chosenCandidate, bestImpurity = impurities[chosenFeature]

        return chosenFeature, chosenCandidate, bestImpurity

    # size is the only hard coded part of the class
    def determinePartitions(self, stepSize=0.025):
        # layer 1
        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(self.trainData, stepSize=stepSize)
        print("\tPartition 0 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[0] = Partition(chosenFeature, chosenCandidate)
        B, C = self.partitions[0](self.trainData)

        # layer 2
        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(B, stepSize=stepSize)
        print("\tPartition 1 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[1] = Partition(chosenFeature, chosenCandidate)
        D, E = self.partitions[1](B)


        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(C, stepSize=stepSize)
        print("\tPartition 2 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[2] = Partition(chosenFeature, chosenCandidate)
        F, G = self.partitions[2](C)

        # layer 3
        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(D, stepSize=stepSize)
        print("\tPartition 3 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[3] = Partition(chosenFeature, chosenCandidate)
        H, I = self.partitions[3](D)


        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(E, stepSize=stepSize)
        print("\tPartition 4 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[4] = Partition(chosenFeature, chosenCandidate)
        J, K = self.partitions[4](E)


        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(F, stepSize=stepSize)
        print("\tPartition 5 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[5] = Partition(chosenFeature, chosenCandidate)
        L, M = self.partitions[5](F)


        chosenFeature, chosenCandidate, bestImpurity = self.findBestPartition(G, stepSize=stepSize)
        print("\tPartition 6 --- Chosen feature: ", chosenFeature, " | Candidate: ", chosenCandidate, " | Impurity: ", bestImpurity)
        self.partitions[6] = Partition(chosenFeature, chosenCandidate)
        N, O = self.partitions[6](G)

        print("\nLeaf nodes: ")
        leaves = [H,I,J,K,L,M,N,O]
        for i, leaf in enumerate(leaves):
            numItems = float(leaf.shape[0])
            numOnes = np.sum(leaf[:, -1])
            if numOnes >= numItems/2:   # 1 is the most popular label
                label = 1
            else:                       # 0 is the most popular label
                label = 0
            print("Leaf ", i, " label: ", label, " | Impurity: ", self.impurity(leaf))

        # print("\tFinal impurity: ", totalFinalImpurity)

    def test(self, items):
        B, C = self.partitions[0](items)
        D, E = self.partitions[1](B)
        F, G = self.partitions[2](C)
        # leaf nodes
        H, I = self.partitions[3](D)
        J, K = self.partitions[4](E)
        L, M = self.partitions[5](F)
        N, O = self.partitions[6](G)

        correctlyPredicted = 0
        totalPredicted = 0

        for i in [H,I,J,K,L,M,N,O]:
            numItems = float(i.shape[0])
            numOnes = np.sum(i[:, -1])
            if numOnes >= numItems/2:   # 1 is the most popular label
                correctlyPredicted += numOnes
            else:                       # 0 is the most popular label
                correctlyPredicted += (numItems - numOnes)

            totalPredicted += numItems

        accuracy = float(correctlyPredicted)/totalPredicted
        return accuracy, correctlyPredicted, totalPredicted

        
data = np.genfromtxt('bankAuth.txt', delimiter=',')
print("Tree generation results: ")

T = DecisionTree(trainData=data)
T.determinePartitions(stepSize=0.01)



Tree generation results: 
Feature:  0  | Best candidate:  -0.21210000000014517  | Impurity:  0.032640888212331694
Feature:  1  | Best candidate:  -6.953100000000145  | Impurity:  0.0576
Feature:  2  | Best candidate:  8.663899999999703  | Impurity:  0.057862729469445075
Feature:  3  | Best candidate:  -8.5482  | Impurity:  0.060975190696247156
	Partition 0 --- Chosen feature:  0  | Candidate:  -0.21210000000014517  | Impurity:  0.032640888212331694
Feature:  0  | Best candidate:  -0.2062  | Impurity:  0.016076188185044096
Feature:  1  | Best candidate:  -6.9321  | Impurity:  0.016076188185044096
Feature:  2  | Best candidate:  -4.966100000000007  | Impurity:  0.014169035610877068
Feature:  3  | Best candidate:  -6.8103  | Impurity:  0.016076188185044096
	Partition 1 --- Chosen feature:  2  | Candidate:  -4.966100000000007  | Impurity:  0.014169035610877068
Feature:  0  | Best candidate:  -7.0421  | Impurity:  0.016564700027287597
Feature:  1  | Best candidate:  9.536899999999505  | Imp

### `sklearn` Decision Tree Experiments

1. Scikit-learn has two decision tree implementations: [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and [`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html). 

When would you use one over the other? What would you use in the case of the banknote authentication dataset? Explain the changes that need to be made in the dataset to use the other implementation.

2. Fit a decision tree to the training set. Change various parameters and compare them to one another. Mainly try and experiment with the `criterion`, `max_depth` and `min_samples_split` parameters. Report the accuracy on the training and test set for each of the experiments while varying the parameters for comparison purposes.

3. Plot your trees !! (optional) (for visualization)

```python
from sklearn.tree import plot_tree

def plotTree(tree):
    """
    tree: Tree instance that is the result of fitting a DecisionTreeClassifier
          or a DecisionTreeRegressor.
    """
    plt.figure(figsize=(30,20))
    plot_tree(tree, filled=True, rounded=True,
                  class_names=['forged', 'authentic'],
                  feature_names=['var', 'skew', 'curt', 'ent'])
    plt.show()
    return None
```

## 1.
You would use a classifier as the model is supposed to output a single class label. A decision tree regressor would produce continuous output which is meaningless for class labels.
You could modify the dataset by labelling each entry with the confidence/probability that a note is forged.

In [144]:
# your code here
data = np.genfromtxt('bankAuth.txt', delimiter=',')
np.random.shuffle(data)

trainItems, testItems = train_test_split(data, test_size=0.3)

clf = DecisionTreeClassifier()

def testTree(clf: DecisionTreeClassifier, trainItems, testItems):
    labels = np.array(trainItems[:,4], dtype=np.int32)
    decision_tree = clf.fit(trainItems[:,:4], labels)

    cp = 0
    tp = testItems.shape[0]
    
    predictions = clf.predict(testItems[:,:4])
    diff = testItems[:,4] - predictions         # exploiting the fact that labels are just 1 or 0

    wp = np.sum(np.abs(diff))
    cp = tp - wp
    
    print("Accuracy: ", float(cp)/tp)
    print(cp, " correctly predicted out of ", tp)

    return float(cp)/tp, cp, tp
        

In [145]:
## Varying criterion,
# max_depth = 3

# gini
print("Gini index: ")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3), trainItems, testItems)

print()
print("Entropy: ")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="entropy", max_depth=3), trainItems, testItems)

print()
print("Log loss: ")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="log_loss", max_depth=3), trainItems, testItems)



Gini index: 
Accuracy:  0.9320388349514563
384.0  correctly predicted out of  412

Entropy: 
Accuracy:  0.9538834951456311
393.0  correctly predicted out of  412

Log loss: 
Accuracy:  0.9538834951456311
393.0  correctly predicted out of  412


In [147]:
## Varying max_depth, using the gini index

print("Depth = 2")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=2), trainItems, testItems)

print("Depth = 3")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3), trainItems, testItems)

print("Depth = 4")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=4), trainItems, testItems)

print("Depth = 5")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=5), trainItems, testItems)

print("Depth = 6")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=6), trainItems, testItems)

print("Depth = 7")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=7), trainItems, testItems)

print("Depth = 8")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=8), trainItems, testItems)



Depth = 2
Accuracy:  0.912621359223301
376.0  correctly predicted out of  412
Depth = 3
Accuracy:  0.9320388349514563
384.0  correctly predicted out of  412
Depth = 4
Accuracy:  0.9611650485436893
396.0  correctly predicted out of  412
Depth = 5
Accuracy:  0.9781553398058253
403.0  correctly predicted out of  412
Depth = 6
Accuracy:  0.9878640776699029
407.0  correctly predicted out of  412
Depth = 7
Accuracy:  0.9878640776699029
407.0  correctly predicted out of  412
Depth = 8
Accuracy:  0.9878640776699029
407.0  correctly predicted out of  412


In [151]:
## Varying min_split

print("Minimum sample = 1")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=2), trainItems, testItems)

print("Minimum sample = 2")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=2), trainItems, testItems)

print("Minimum sample = 5")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=5), trainItems, testItems)

print("Minimum sample = 10")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=10), trainItems, testItems)

print("Minimum sample = 25")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=25), trainItems, testItems)

print("Minimum sample = 50")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=50), trainItems, testItems)

print("Minimum sample = 100")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=100), trainItems, testItems)

print("Minimum sample = 200")
acc, cp, tp = testTree(DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_split=200), trainItems, testItems)

Minimum sample = 2
Accuracy:  0.9320388349514563
384.0  correctly predicted out of  412
Minimum sample = 5
Accuracy:  0.9320388349514563
384.0  correctly predicted out of  412
Minimum sample = 10
Accuracy:  0.9320388349514563
384.0  correctly predicted out of  412
Minimum sample = 25
Accuracy:  0.9320388349514563
384.0  correctly predicted out of  412
Minimum sample = 50
Accuracy:  0.9271844660194175
382.0  correctly predicted out of  412
Minimum sample = 100
Accuracy:  0.912621359223301
376.0  correctly predicted out of  412
Minimum sample = 200
Accuracy:  0.912621359223301
376.0  correctly predicted out of  412


### Load Data

The data has been loaded onto a Pandas DataFrame. Try to get an initial feel for the data by using functions like `describe()`, `info()`, or maybe try to plot the data to check for any patterns.

Note: To obtain the data from the UCI website, `wget` can be used followed by shuffling the samples using `shuf` and adding a header for easier reading via `pandas`. It is not necessary to view the data in a DataFrame and can be directly loaded onto NumPy as convenient.

In [100]:
data = np.genfromtxt('bankAuth.txt', delimiter=',')
np.random.shuffle(data)


### Splitting the Data

It is a good practice to split the data into training and test sets. This is to ensure that the model is not overfitting to the training data. The test set is used to evaluate the performance of the model on unseen data. The test set is not used to train the model in any way. The test set is only used to evaluate the performance of the model. You may use the `train_test_split` function from `sklearn.model_selection` to split the data into training and test sets.

It is a good idea to move your data to NumPy arrays now as it will make computing easier.

In [120]:
# your code here
trainItems, testItems = train_test_split(data, test_size=0.3)

### Denouement

Use this place to report all comparisons and wrap up the calls to the functions written above.

In [125]:
# your code here
T = DecisionTree(trainData=trainItems)

print("Tree generation results: ")
T.determinePartitions(stepSize=0.0025)

accuracy, cp, tp = T.test(testItems)
print("Leaf node accuracy: ", accuracy)
print(cp, " correctly predicted out of ", tp)

Tree generation results: 
Feature:  0  | Best candidate:  -0.21639999999893433  | Impurity:  0.03213309164899718
Feature:  1  | Best candidate:  -6.880600000001371  | Impurity:  0.0572571034167722
Feature:  2  | Best candidate:  8.65640000000218  | Impurity:  0.05761308759776604
Feature:  3  | Best candidate:  -7.8719  | Impurity:  0.06120416146855296
	Partition 0 --- Chosen feature:  0  | Candidate:  -0.21639999999893433  | Impurity:  0.03213309164899718
Feature:  0  | Best candidate:  -0.2062  | Impurity:  0.016094172540127037
Feature:  1  | Best candidate:  -6.9321  | Impurity:  0.016094172540127037
Feature:  2  | Best candidate:  -4.973599999999951  | Impurity:  0.014445191109309332
Feature:  3  | Best candidate:  -6.8103  | Impurity:  0.016094172540127037
	Partition 1 --- Chosen feature:  2  | Candidate:  -4.973599999999951  | Impurity:  0.014445191109309332
Feature:  0  | Best candidate:  -7.0364  | Impurity:  0.01603891910887014
Feature:  1  | Best candidate:  9.60189999999535  